In [1]:
import gym
import ptan
import random

import torch
import torch.optim as optim

from ignite.engine import Engine

from lib import common, dqn_extra

NAME = "04_noisy"
NOISY_SNR_EVERY_ITERS = 100

In [2]:
random.seed(common.SEED)
torch.manual_seed(common.SEED)
params = common.HYPERPARAMS['pong']
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

env = gym.make("PongNoFrameskip-v4")
env = ptan.common.wrappers.wrap_dqn(env)
env.seed(common.SEED)

[123, 151010689]

In [3]:
net = dqn_extra.NoisyDQN(env.observation_space.shape, env.action_space.n).to(device)

tgt_net = ptan.agent.TargetNet(net)
selector = ptan.actions.ArgmaxActionSelector()
agent = ptan.agent.DQNAgent(net, selector, device = device)

exp_source = ptan.experience.ExperienceSourceFirstLast(
    env, agent, gamma=params.gamma)
buffer = ptan.experience.ExperienceReplayBuffer(
    exp_source, buffer_size=params.replay_size)
optimizer = optim.Adam(net.parameters(), lr=params.learning_rate)

In [4]:
def process_batch(engine, batch):
    optimizer.zero_grad()
    loss_v = common.calc_loss_dqn(batch, net, tgt_net.target_model,
                                  gamma=params.gamma, device=device)
    loss_v.backward()
    optimizer.step()
    if engine.state.iteration % params.target_net_sync == 0:
        tgt_net.sync()
    if engine.state.iteration % NOISY_SNR_EVERY_ITERS == 0:
        for layer_idx, sigma_l2 in enumerate(net.noisy_layers_sigma_snr()):
            engine.state.metrics[f'snr_{layer_idx+1}'] = sigma_l2
    return {
        "loss": loss_v.item(),
    }

engine = Engine(process_batch)
common.setup_ignite(engine, params, exp_source, NAME, extra_metrics=('snr_1', 'snr_2'))
engine.run(common.batch_generator(buffer, params.replay_initial, params.batch_size))

Episode 1: reward=-21, steps=818, speed=0.0 f/s, elapsed=0:01:05
Episode 2: reward=-21, steps=819, speed=0.0 f/s, elapsed=0:01:05
Episode 3: reward=-21, steps=761, speed=0.0 f/s, elapsed=0:01:05
Episode 4: reward=-21, steps=757, speed=0.0 f/s, elapsed=0:01:05
Episode 5: reward=-21, steps=818, speed=0.0 f/s, elapsed=0:01:05
Episode 6: reward=-21, steps=758, speed=0.0 f/s, elapsed=0:01:05
Episode 7: reward=-21, steps=758, speed=0.0 f/s, elapsed=0:01:05
Episode 8: reward=-21, steps=762, speed=0.0 f/s, elapsed=0:01:05
Episode 9: reward=-21, steps=762, speed=0.0 f/s, elapsed=0:01:05
Episode 10: reward=-21, steps=759, speed=0.0 f/s, elapsed=0:01:05
Episode 11: reward=-21, steps=759, speed=0.0 f/s, elapsed=0:01:05
Episode 12: reward=-21, steps=758, speed=0.0 f/s, elapsed=0:01:05
Episode 13: reward=-21, steps=760, speed=53.6 f/s, elapsed=0:01:06
Episode 14: reward=-21, steps=776, speed=53.9 f/s, elapsed=0:01:18
Episode 15: reward=-21, steps=818, speed=54.1 f/s, elapsed=0:01:30
Episode 16: rewa

Engine run is terminating due to exception: .


KeyboardInterrupt: 